In [1]:
# Projeto Marinha do Brasil

# Autor: Natanael Junior (natmourajr@gmail.com)
# Laboratorio de Processamento de Sinais - UFRJ

In [1]:
import os
import pickle
import numpy as np
import time

init_time = time.time()

from sklearn import preprocessing
from sklearn import cross_validation
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.externals import joblib

m_time = time.time()
print 'Time to import all libraries: '+str(m_time-init_time)+' seconds'

outputpath = os.environ['OUTPUTDATAPATH']
main_analysis_path = os.environ['SONAR_WORKSPACE']
log_analysis_path = os.environ['PACKAGE_OUTPUT']
result_analysis_path = os.environ['PACKAGE_OUTPUT']+'/PCDSingleClassSVM'
# Read data
# Check if LofarData has created...
m_time = time.time()


subfolder = '4classes'
n_pts_fft = 1024
decimation_rate = 3

if(not os.path.exists(outputpath+'/'+'LofarData_%s_%i_fft_pts_%i_decimation_rate.pickle'%(
            subfolder,n_pts_fft,decimation_rate))):
    print outputpath+'/'+'LofarData_%s_%i_fft_pts_%i_decimation_rate.pickle'%(
        subfolder,n_pts_fft,decimation_rate)+' doesnt exist...please create it'
    exit()
    
#Read lofar data
[data,class_labels] = pickle.load(open(outputpath+'/'+'LofarData_%s_%i_fft_pts_%i_decimation_rate.pickle'%(
            subfolder,n_pts_fft,decimation_rate), "rb" ))
m_time = time.time()-m_time
print 'Time to read data file: '+str(m_time)+' seconds'

Time to import all libraries: 4.51343822479 seconds
Time to read data file: 7.43266487122 seconds


# Train process
## The train will modify one file and create three different files

### Log File:
This file will store basic information of all Package's trains and it will guide the analyses file to recognize which train information file should load. In each train this file should be appended with a new line contend the basic information to find the train information file (TXT FORMAT) or (PYTHON FORMAT) - This file should be access by all programs (MatLab and Python) for Analysis

### Train Information File
This file will store full information of the train performed (all parameters) in its name (each train information file will have a different name). And it will guide which train classifier file or which train result file should be open for analysis (TXT FORMAT) or (PYTHON FORMAT) - This file should be access by all programs (MatLab and Python) for Analysis

### Train Classifier File
This file will store the classifier after train, the folds information and the train evolution (when this exists) (NATIVE FORMAT) or (PYTHON FORMAT) - This file should not be access by all programs

### Train Result File
This file will store the classifier result for all data and classification target (TXT FORMAT) or (PYTHON FORMAT) - This file should be access by all programs (MatLab and Python) for Analysis

### Train Preprocessing File
This file will store the preprocesing after train, the folds information and the train evolution (when this exists) (NATIVE FORMAT) or (PYTHON FORMAT) - This file should not be access by all programs



In [2]:
# Train Process
from Functions import LogFunctions as log
from Functions import MultiLayerPerceptron as mlpLib

# to be deleted!!!
import time
date = time.strftime("%Y_%m_%d_%H_%M_%S")

# Create a entry in log file
#m_log = log.LogInformation()
#date = m_log.CreateLogEntry("NoveltyDetection",'PCDSingleClassSVM')

# Create a train information file
n_folds = 5
n_pcds = 2  # number of PCD to be extracted
norm = 'mapstd'

train_info = {}
train_info['n_folds'] = n_folds
train_info['n_pcds'] = n_pcds
train_info['norm'] = norm

train_info_name = result_analysis_path+'/train_info_files'+'/'+date+'_preprocessing_train_info.jbl'
class_info_name = result_analysis_path+'/classifiers_files'+'/'+date+'_preprocessing.jbl'

# Collecting data
all_data = {};
all_trgt = {};

class_ids = []

for iclass, class_label in enumerate(class_labels):
    class_ids = np.append(class_ids, iclass)
    for irun in range(len(data[iclass])):
        if len(all_data) == 0:
            all_data = data[iclass][irun]['Signal']
            all_trgt = (iclass)*np.ones(data[iclass][irun]['Signal'].shape[1])
        else:
            all_data = np.append(all_data,data[iclass][irun]['Signal'],axis=1)
            all_trgt = np.append(all_trgt,(iclass)*np.ones(data[iclass][irun]['Signal'].shape[1]),axis=0)

all_data = all_data.transpose()

for novelty_class, novelty_label in enumerate(class_labels):
    train_data = all_data[all_trgt!=novelty_class,:]
    train_trgt = all_trgt[all_trgt!=novelty_class]
    
    # creating max sparse trgt
    sparse_all_trgt = np.zeros([train_data.shape[0],len(class_labels)-1])
    for iclass, class_label in enumerate(class_labels):
        if iclass == novelty_class: continue
        sparse_all_trgt[train_trgt==iclass,iclass-(iclass>novelty_class)] = 1
        
        
    # Cross validation
    CVO = cross_validation.StratifiedKFold(train_trgt, n_folds)
    CVO = list(CVO)
    train_info['CVO_%s'%(novelty_label)] = CVO
    
    for ifold in range(len(CVO)):
        print '\nNovelty: %s - ifold %i of %i'%(novelty_label,ifold,len(CVO))
        # split data in trn set, tst set
        train_id, test_id = CVO[ifold]
        
    
joblib.dump([train_info],train_info_name,compress=9)

Novelty: ClassA - ifold 0 of 5
Train Function
Starting Train Process with the follow parameters:


/Users/natmourajr/Workspace/Doutorado/SonarAnalysis/Functions/MultiLayerPerceptron.py:528: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if self.trn_params.itrn == None:




Training...
Epoch 0200 - trn Cost: 0.069 - tst Cost: 0.209 - val Cost: 0.209 - Best Epoch: 0200 - val Cost: 0.209
Final Epoch  254  - trn Cost:  0.0689622851631
Novelty: ClassB - ifold 0 of 5
Train Function
Starting Train Process with the follow parameters:


Training...
Epoch 0200 - trn Cost: 0.079 - tst Cost: 0.154 - val Cost: 0.154 - Best Epoch: 0200 - val Cost: 0.154
Final Epoch  358  - trn Cost:  0.0769199400105
Novelty: ClassC - ifold 0 of 5
Train Function
Starting Train Process with the follow parameters:


Training...
Epoch 0200 - trn Cost: 0.073 - tst Cost: 0.164 - val Cost: 0.164 - Best Epoch: 0200 - val Cost: 0.164
Epoch 0400 - trn Cost: 0.072 - tst Cost: 0.160 - val Cost: 0.160 - Best Epoch: 0400 - val Cost: 0.160
Epoch 0600 - trn Cost: 0.072 - tst Cost: 0.158 - val Cost: 0.158 - Best Epoch: 0600 - val Cost: 0.158
Epoch 0800 - trn Cost: 0.072 - tst Cost: 0.158 - val Cost: 0.158 - Best Epoch: 0800 - val Cost: 0.158
Epoch 1000 - trn Cost: 0.072 - tst Cost: 0.158 - val Cost:

['/Users/natmourajr/Workspace/Doutorado/SonarAnalysis/Results/NoveltyDetection/PCDSingleClassSVM/train_info_files/2016_07_15_17_57_42_preprocessing_train_info.jbl']

In [77]:
print train_data.shape[1]
print train_data.shape

(17926, 3)
(17926, 400)
